# Bernstein polynomials based coloring

This notebook generates color channel charts and colorbars based on Bernstein polynomials that are used to derive RGB colors from the escape time that was calculated for different fractals. Please note this notebook uses fake escape times.

In [102]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as col
import matplotlib.cm as cm

In [103]:
def rgb_continuous_bernstein(escape_time, bailout, rgb_base, rgb_amp):
    '''
    Convert cont_index with rgb_base rgb_freq and rgb_phase to a rgb value between 0 and 255 
    using adapted Bernstein polynomial
    '''
    red = rgb_base[0]
    green = rgb_base[1]
    blue = rgb_base[2]
    if (escape_time == bailout):
        return (0, 0, 0)
    
    index_mapped = escape_time / bailout
    
    if (rgb_amp[0] > 0):
        red = np.abs(rgb_amp[0] * (1 - index_mapped) * np.power(index_mapped, 3) * (255 - rgb_base[0]) + rgb_base[0] )
    if (rgb_amp[1] > 0):
        green = np.abs(rgb_amp[1] * np.power((1 - index_mapped), 2) * np.power(index_mapped, 2 ) * (255 - rgb_base[1]) + rgb_base[1])
    if (rgb_amp[2] > 0):
        blue = np.abs(rgb_amp[2] * np.power((1 - index_mapped), 3) * index_mapped * (255 - rgb_base[2]) + rgb_base[2])
    
    rgb = np.array([red, green, blue])
    return rgb.astype(int)

In [104]:
def generate_rgb_tuples(escape_idx, bailout, rgb_base, rgb_amp):
    '''
    Fill arrays with rgb tuples
    '''
    rgb_tuples = np.zeros((escape_idx.size,3), dtype='uint8')
    tuples_idx = 0
    
    for etime in escape_idx:
        rgb_tuples[tuples_idx] = rgb_continuous_bernstein(etime,
                                                bailout,
                                                rgb_base,
                                                rgb_amp)
        # print(rgb_tuples[tuples_idx])
        tuples_idx +=1
        
    return rgb_tuples

In [105]:
escape_idx = np.linspace(0, 1000, 1001)
rgb_tuples_1 = generate_rgb_tuples(escape_idx, 
                                   escape_idx.size,
                                   (0, 0, 0),
                                   # you have to adjust the frequency so it corresponds with the fractal bailout
                                   (9, 15, 8.5))

In [106]:
bounds = np.arange(0, 1000, 1)
norm_obj_1 = col.Normalize(vmin=0,
                           vmax=255)

In [107]:
colmap_1 = col.ListedColormap(norm_obj_1(rgb_tuples_1))
print(colmap_1)

In [108]:
norm_1 = mpl.colors.BoundaryNorm(bounds, colmap_1.N)

In [109]:
fig = plt.figure(1, figsize=(10, 6))
fig.suptitle("Modified Bernstein polynomials", fontsize='medium', weight='bold')
ax_1 = fig.add_subplot(111)
ax_1.plot(escape_idx, rgb_tuples_1.T[0], c='red', lw=2)
ax_1.plot(escape_idx, rgb_tuples_1.T[1], c='green', lw=2)
ax_1.plot(escape_idx, rgb_tuples_1.T[2], c='blue', lw=2)
ax_1.set_xlabel("Continuous Index")
ax_1.set_ylabel("Color Channel")
rgb_settings_1 = "rgb-base=0,0,0 rgb-amp=9,15,8.5"
ax_1.text(0.5, 0.92, rgb_settings_1,
         horizontalalignment='center',
         fontsize='small',
          weight='bold',
         transform = ax_1.transAxes)
ax_1.grid()
# make room for an additional axis at the bottom
fig.subplots_adjust(top=0.93, bottom=0.27)
# get the bounding box
box = ax_1.get_position()
# use the box to position the colorbar
ax_colb = fig.add_axes([box.x0, box.y0 - 0.18, box.width, 0.1])
cb_1 = mpl.colorbar.ColorbarBase(ax_colb, cmap=colmap_1,
                                 norm=norm_1,
                                 boundaries=bounds,
                                 # extend='both',
                                 # Make the length of each extension
                                 # the same as the length of the
                                 # interior colors:
                                 extendfrac='auto',
                                 spacing='uniform',
                                 orientation='horizontal')
cb_1.set_label("Color spectrum from black to black")

In [110]:
plt.show()